In [1]:
import networkx as nx;
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def solver_optimistic (G, budget, s, t, A):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';
    
    opt_model = gp.Model("Optimistic"); 
    
    gamma = opt_model.addVars(G.edges, vtype=GRB.BINARY, name = "gamma"); 
    z = opt_model.addVars(G.edges, vtype=GRB.BINARY, name = "z"); 
    x = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "x", lb = 0, ub = GRB.INFINITY);
    theta = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "theta", lb = 0, ub = 1); 
    alpha = opt_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name = "alpha", lb = 0, ub = 1); 
    beta = opt_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "beta", lb = 0, ub = 1); 
    
    
    opt_model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    opt_model.addConstr(gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges) <= x[t,s]);
    
    opt_model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            opt_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*z[i,j] <= 0);
        
        opt_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        opt_model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        opt_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    opt_model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
            
    opt_model.setObjective(gp.quicksum(G.edges[i,j]['special']*z[i,j] for i,j in G.edges), GRB.MINIMIZE);
    
    opt_model.setParam("IntegralityFocus",1);
    #opt_model.setParam('TimeLimit', T_Limit); 
    opt_model.update();
    
    opt_model.setParam("OutputFlag", 0);
    opt_model.optimize();
    
    
    status = opt_model.status;
    end_time = time.time();
    
    print('Optimistic LB: %g' %opt_model.objVal);
    run_time = round(end_time - start_time, 2);
    print('run time: %g sec \n' %run_time);
    
    # ~~~~~~~~~~~~~~~~~ OPTIMISTIC MIP START FILE~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    mst_file = open('MST_Optimistic_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    return opt_model.objVal;

In [3]:
def solver_MFNIP (budget, network):
    
    start_time = time.time();
    networkCSV = 'Modified_'+network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);  
   
    MFNIP_model = gp.Model("MFNIP"); 
    
    gamma = MFNIP_model.addVars(G.edges, vtype=GRB.BINARY, name="gamma"); 
    x = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="x", lb = 0, ub = GRB.INFINITY);
    theta = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="theta", lb = 0, ub = 1); 
    alpha = MFNIP_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="alpha", lb = 0, ub = 1); 
    beta = MFNIP_model.addVars(G.edges, vtype=GRB.CONTINUOUS, name="beta", lb = 0, ub = 1); 
    
    
    MFNIP_model.addConstr(gp.quicksum(G.edges[i,j]['cost']*gamma[i,j] for i,j in G.edges) <= budget);
    
    MFNIP_model.addConstr(x[t,s] >= gp.quicksum(G.edges[i,j]['capacity']*beta[i,j] for i,j in G.edges));
    
    MFNIP_model.addConstr(alpha[t] - alpha[s] >= 1);
    
    for i,j in G.edges:
        MFNIP_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
        MFNIP_model.addConstr(beta[i,j] + gamma[i,j] - theta[i,j] >= 0);
        MFNIP_model.addConstr(x[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j]) <= 0);
            
    MFNIP_model.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) -
                     gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
     
    
    MFNIP_model.setObjective(x[t,s], GRB.MINIMIZE);
    
    MFNIP_model.setParam("IntegralityFocus",1);
    MFNIP_model.update();

    MFNIP_model.setParam("OutputFlag", 0);
    MFNIP_model.optimize();
    
    
    # ~~~~~~~~~~~~ MST FILE (ONLY GAMMA VALUES) ~~~~~~~~~~~~~~~~~~~~~~  ##
    mst_file = open('MST_MFNIP_'+network+'_b'+str(budget)+'.mst', "w");
    
    mst_file.write('# gamma values \n');
    for i,j in G.edges:
        mst_file.write('%s %g \n' %(gamma[i,j].VarName, gamma[i,j].x));
    mst_file.write('#\n');
    
    mst_file.close();
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ##
    
    end_time = time.time();
    print('MFNIP LB: %g' %MFNIP_model.objVal);
    run_time = round(end_time - start_time, 2);
    print('run time: %g sec \n' %run_time);
    
    #return MFNIP_model.objVal;

In [4]:
# Max Problem (to obtain the values of x and z given an interdiction)

def InnerProblem (Gamma, G, s, t, M):
    
  
    max_model = gp.Model("Max_z");
    
    x = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = max_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sum_z = max_model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = M);
            
    max_model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                         - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    for i,j in G.edges:
        max_model.addConstr(x[i,j] <= (1-Gamma[i,j])*G.edges[i,j]['capacity']);
            
    for i,j in G.edges:
        max_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
            
    max_model.addConstr(alpha[t] - alpha[s] >= 1);
            
    max_model.addConstr(x[t,s] >= gp.quicksum((1-Gamma[i,j])*G.edges[i,j]['capacity']*theta[i,j]
                                                     for i,j in G.edges));
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            max_model.addConstr(x[i,j] >= (1/M)*z[i,j]);
            
    max_model.addConstr(sum_z <= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
    max_model.setObjective(sum_z, GRB.MAXIMIZE); 
            
    max_model.setParam("IntegralityFocus",1);
    #max_model.setParam("NumericFocus",2);       
    #max_model.setParam('TimeLimit', T_Lim);
            
    max_model.update();
    max_model.setParam("OutputFlag", 0);
    max_model.optimize();
    
    if max_model.SolCount == 0:
        print("No feasible solution available (Max-Problem)");
    
    
    X = {};
    Z = {};
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Z[i,j] = z[i,j].x;
      
    return X, Z;

In [5]:
def Dual_Inner (Gamma, G, s, t, M):
    
    Mo = 10e6;
    
    for i,j in G.edges:
        if Gamma[i,j] >= 0.001:
            print('gamma[%g,%g] = %g' %(i,j,Gamma[i,j]));
    
    dualModel = gp.Model("dual_Inner");
    
    
    chi = dualModel.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="chi", lb = -GRB.INFINITY, ub = GRB.INFINITY);
    omega = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="omega", lb = 0, ub = GRB.INFINITY);
    delta = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="delta", lb = -GRB.INFINITY, ub = 0);
    phi = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="phi", lb = -GRB.INFINITY, ub = 0);   
    epsilon = dualModel.addVars(G.nodes, vtype=GRB.CONTINUOUS, name="epsilon", lb = 0, ub = GRB.INFINITY);
    nu = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="nu", lb = 0, ub = GRB.INFINITY);
    kappa = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="kappa", lb = 0, ub = GRB.INFINITY);
    
    psi = dualModel.addVars(G.edges, vtype=GRB.CONTINUOUS, name="psi", lb = 0, ub = GRB.INFINITY);
    
    eta = dualModel.addVar(vtype=GRB.CONTINUOUS, name="eta", lb = -GRB.INFINITY, ub = 0);
    pi = dualModel.addVar(vtype=GRB.CONTINUOUS, name="pi", lb = -GRB.INFINITY, ub = 0);
    
    val = dualModel.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY);
    
    dualModel.addConstr(val >= gp.quicksum(G.edges[i,j]['capacity']*(1-Gamma[i,j])*omega[i,j] for i,j in G.edges) +
                    pi + gp.quicksum(epsilon[i] for i in G.nodes) + gp.quicksum(nu[i,j] for i,j in G.edges) + 
                    gp.quicksum(kappa[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
    
    
    dualModel.addConstr(chi[t] - chi[s] + omega[t,s] + eta >= 0);
    
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            dualModel.addConstr(chi[i] - chi[j] + omega[i,j] + phi[i,j] >= 0);
        elif (i != t and j!= s):
            dualModel.addConstr(chi[i] - chi[j] + omega[i,j] >= 0);
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            dualModel.addConstr(kappa[i,j] - (1/M)*phi[i,j] >= 1);
            
    dualModel.addConstr(gp.quicksum(delta[t,j] for j in G.successors(t))-
                    gp.quicksum(delta[j,t] for j in G.predecessors(t)) + epsilon[t] + pi >= 0);
            
    dualModel.addConstr(gp.quicksum(delta[j,s] for j in G.predecessors(s))-
                    gp.quicksum(delta[s,j] for j in G.successors(s)) + epsilon[s] - pi >= 0);
    
    for i in G.nodes:
        if i != s and i != t:
            dualModel.addConstr(gp.quicksum(delta[i,j] for j in G.successors(i))-
                            gp.quicksum(delta[j,i] for j in G.predecessors(i)) + epsilon[i] >= 0);
                    
    for i,j in G.edges:
        dualModel.addConstr(delta[i,j] - G.edges[i,j]['capacity']*(1-Gamma[i,j])*eta + nu[i,j] >= 0);

        
    dualModel.setObjective(val, GRB.MINIMIZE); 
    dualModel.update();
    
    dualModel.setParam("OutputFlag", 0);
    dualModel.optimize();

In [6]:
def solver_Pessimistic (network, budget, T_Limit, summaryName):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);  
    
    
    A = 0;
    U = G.edges[t,s]['capacity'];
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
    
    M = A;
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###
    
    ### ~~~~~~~~~~~~~~~~~~~~ SOLVING OPTIMISTIC PROBLEM (Optmistic LB and MIP Start)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ####
    LB_opt = solver_optimistic (G, budget, s, t, A);
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ####
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~ SOLVING MFNIP METHOD (MFNIP MIP start) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ###
    solver_MFNIP (budget, network);
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    
    
    ### Optimization Model ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
    
    instance = 'LogFile_'+network+'_b'+str(budget)+'.txt';
    
    model = gp.Model("Pessimistic_EarlyRelax");  
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY, name = "gamma");
    chi = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name = "chi", lb = -GRB.INFINITY, ub = GRB.INFINITY);
    omega = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "omega", lb = 0, ub = GRB.INFINITY);
    phi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "phi", lb = -GRB.INFINITY, ub = 0);
    eta = model.addVar(vtype=GRB.CONTINUOUS, name = "eta", lb = -GRB.INFINITY, ub = 0);
    kappa = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "kappa", lb = 0, ub = GRB.INFINITY);
    delta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "delta", lb = -GRB.INFINITY, ub = 0);
    epsilon = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, name = "epsilon", lb = 0, ub = GRB.INFINITY);
    pi = model.addVar(vtype=GRB.CONTINUOUS, name = "pi", lb = -GRB.INFINITY, ub = 0);
    nu = model.addVars(G.edges, vtype=GRB.CONTINUOUS,name = "nu", lb = 0, ub = GRB.INFINITY);
    
    psi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "psi", lb = 0, ub = GRB.INFINITY);
    xi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, name = "xi", lb = -GRB.INFINITY, ub = 0);
    
    val = model.addVar(vtype=GRB.CONTINUOUS, lb = LB_opt, ub = A);
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    
    ### Constraints ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    ## WARM START WITH OPTIMISTIC AND MFNIP INTERDICTION PLANS ###
    
    model.NumStart = 2;
    model.update();
    
    # iterate over all MIP starts
    for start in range(model.NumStart):

        # set StartNumber
        model.params.StartNumber = start;
        
        if start == 0:
            plan = 'MST_MFNIP_'+network+'_b'+str(budget)+'.mst';
            model.read(plan);
        
        elif start == 1:
            plan = 'MST_Optimistic_'+network+'_b'+str(budget)+'.mst';
            model.read(plan);
             
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #                
        
    
    
    model.addConstr(val >= gp.quicksum(omega[i,j]*(1-gamma[i,j])*G.edges[i,j]['capacity'] for i,j in G.edges) +
                    pi + gp.quicksum(epsilon[i] for i in G.nodes) + gp.quicksum(nu[i,j] for i,j in G.edges) + 
                    gp.quicksum(kappa[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
    
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget);
            
    model.addConstr(chi[t] - chi[s] + omega[t,s] + eta >= 0);
    
    
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            model.addConstr(chi[i] - chi[j] + omega[i,j] + phi[i,j] >= 0);
        elif (i != t and j!= s):
            model.addConstr(chi[i] - chi[j] + omega[i,j] >= 0);
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            model.addConstr(kappa[i,j] - (1/M)*phi[i,j] >= 1);
            
    model.addConstr(gp.quicksum(delta[t,j] for j in G.successors(t))-
                    gp.quicksum(delta[j,t] for j in G.predecessors(t)) + epsilon[t] + pi >= 0);
            
    model.addConstr(gp.quicksum(delta[j,s] for j in G.predecessors(s))-
                    gp.quicksum(delta[s,j] for j in G.successors(s)) + epsilon[s] - pi >= 0);
    
    for i in G.nodes:
        if i != s and i != t:
            model.addConstr(gp.quicksum(delta[i,j] for j in G.successors(i))-
                            gp.quicksum(delta[j,i] for j in G.predecessors(i)) + epsilon[i] >= 0);
                    
    for i,j in G.edges:
        model.addConstr(delta[i,j] - G.edges[i,j]['capacity']*(1-gamma[i,j])*eta + nu[i,j] >= 0);

    
    
        
    model.setObjective(val, GRB.MINIMIZE); 
    model.setParam("IntegralityFocus",1);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("LogToConsole", 0);
    model.setParam("OutputFlag", 1);
    model.setParam("LogFile", instance);
    model.optimize();
    
    status = model.status;
    
    Dual_ObjVal = 0;
    Gamma = {};
    for i,j in G.edges:
        Gamma[i,j] = gamma[i,j].x;
        
    ### Solving auxiliary (inner) problem ###
    
    x, z = InnerProblem (Gamma, G, s, t, M);
    
    ##########################################
    
    
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('EarlyRelax_'+network+'_b'+str(budget)+'.txt', "w");
    file.write('Instance: %s, budget %g \n' %(network, budget));
    file.write('Instance executed at: %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g \n' % (t+1));
    file.write('Number of Arcs: %g \n' % G.number_of_edges());
    file.write('Number of Special Arcs: %g \n' %A);
    file.write('Level of special set: %g \n' % calA_level);
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n');
    print('Instance: %s, budget %g' %(network, budget));
    
    
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g', status);
        file.write('Status: %g \n', status);
    
    
    
    # ~~~~~~ DUAL INNER PROBLEM ~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
    #print('\nDual InnerProblem: Obj Value = %g\n' %Dual_ObjVal);
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    
    print('\nMax-Flow: %g' %x[t,s]);
    file.write('Max-Flow: %g \n' %x[t,s]);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    file.write('Level 0 Capacities: %g \n\n ' %G.edges[s,1]['capacity']);
    
    print('Intediction:');
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    
    lev1 = 0;
    lev2 = 0;
    lev3 = 0;
    other_level = 0;
    
    file.write('\nInterdiction plan: \n');
    
    key1 = False;
    key2 = False;
    key3 = False;
    
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001:
            if G.edges[i,j]["level"] == 1:
                if key1 == False:
                    file.write("Level 1:" +'\n');
                    key1 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev1 += 1;
            elif G.edges[i,j]["level"] == 2:
                if key2 == False:
                    file.write('Level 2: \n');
                    key2 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev2 += 1;
            elif G.edges[i,j]["level"] == 3:
                if key3 == False:
                    file.write('Level 3: \n');
                    key3 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev3 += 1;
            else:
                file.write('Other Levels: \n');
                file.write("arc (%g,%g), gamma = %g\n" %(i,j, gamma[i,j].x));
                other_level += 1;
                
    file.write('\nSpecial arcs with flow : \n');    
    print('Special arcs with flow : \n');
    
    for i, j in G.edges:
        if x[i,j] > 0.00001:
            if G.edges[i,j]['special'] == 1:
                file.write('Arc (%s,%s), Flow: %f \n' %(i,j, x[i,j]));
                print('Arc (%s,%s), Flow: %f ' %(i,j, x[i,j]));
    
    print('\n');

    
    
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, calA_level, (t+1), G.number_of_edges(), A, OptGap, round(obj), x[t,s],
                 lev1, lev2, lev3, other_level, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();


In [ ]:
n_Networks = 10;
Budget = [0,3,6,9,12];

T_Limit = 7200;


summaryName = "EarlyRelax_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance,Budget,Cal_A,Nodes,Arcs,SpecialArcs,OptGap,');
file_summary.write('ObjVal, MaxFlow, Int_L1, Int_L2, Int_L3,Other_Lev,Time\n');
file_summary.close();


for n in range(1, n_Networks+1):
    network = 'Net'+str(n);
    
    for budget in Budget:
        solver_Pessimistic(network, budget, T_Limit, summaryName);

Set parameter TokenServer to value "license8.clemson.edu"
Set parameter IntegralityFocus to value 1
Optimistic LB: 12
run time: 0.13 sec 

Set parameter IntegralityFocus to value 1
MFNIP LB: 16
run time: 0.02 sec 

Read MIP start from file MST_MFNIP_Net1_b0.mst
Read MIP start from file MST_Optimistic_Net1_b0.mst
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 7200
Set parameter IntegralityFocus to value 1


Instance: Net1, budget 0
Optimistic LB: 12 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 16

Max-Flow: 286
run time: 0.27 sec 

Intediction:


Special arcs with flow : 

Arc (1,6), Flow: 21.000000 
Arc (1,7), Flow: 10.937500 
Arc (1,8), Flow: 30.000000 
Arc (2,6), Flow: 16.000000 
Arc (2,7), Flow: 16.000000 
Arc (2,8), Flow: 16.937500 
Arc (2,9), Flow: 9.000000 
Arc (3,8), Flow: 21.000000 
Arc (3,9), Flow: 27.000000 
Arc (3,10), Flow: 25.000000 
Arc (4,6), Flow: 18.000000 
Arc (4,7), Flow: 0.062500 
Arc (4,9), F